# Project 6: Secure Chat App

# Problem Statement:
In an era where digital communication is paramount, the privacy and security of messages are often compromised due to the lack of default end-to-end encryption in many mainstream chat applications. This vulnerability exposes sensitive information to potential eavesdropping and interception.

# Objective:
The primary objective of this project is to develop a rudimentary secure messaging application that facilitates encrypted text communication between two users. This application will serve as a proof-of-concept for secure data exchange over a network, emphasizing the implementation of cryptographic principles for privacy.

# Technologies Utilized:
* **Python:** The core programming language for the application logic and network communication.
* **Socket Programming (`socket` module):** Essential for establishing network connections and handling data transmission between the chat participants (client and server).
* **PyCryptodome (`Crypto.PublicKey.RSA`, `Crypto.Cipher.PKCS1_OAEP`):** This robust cryptographic library will be used to implement RSA asymmetric encryption. This allows for secure key exchange and subsequent message encryption/decryption, ensuring end-to-end security.
* **Threading (`threading` module):** To handle simultaneous sending and receiving of messages without blocking the application.

# Expected Outcome:
A basic yet functional command-line based chat tool capable of providing end-to-end encrypted text communication. This tool will demonstrate the core principles of secure messaging and is suitable for educational purposes and internal demonstrations of secure communication protocols.

In [1]:
!pip install pycryptodome

import socket
import threading
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import base64 # For encoding/decoding encrypted bytes to be sent over network

print("Required libraries installed and imported.")

Required libraries installed and imported.


# Key Generation and Exchange Functions

For end-to-end encryption, each participant needs their own pair of RSA public and private keys. Public keys are exchanged, allowing each participant to encrypt messages that only the other participant (who holds the corresponding private key) can decrypt.

* **`generate_keys()`:** Creates a new RSA public/private key pair.
* **`exchange_keys(sock, my_public_key)`:** Handles the sending of public key and receiving of the other participant's public key over the established socket connection.

In [3]:
def generate_keys():
    """Generates an RSA public and private key pair."""
    key = RSA.generate(2048) # Generate a 2048-bit RSA key pair
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    print("RSA key pair generated.")
    return private_key, public_key

def exchange_keys(sock, my_public_key_pem):
    """
    Exchanges public keys with the other participant over the socket.
    my_public_key_pem: Your public key in PEM format (bytes).
    Returns the other participant's public key as a PyCryptodome RSA key object.
    """
    # Send my public key
    sock.sendall(my_public_key_pem)
    print("My public key sent.")

    # Receive other participant's public key
    other_public_key_pem = sock.recv(2048) # Assuming key size is max 2048 bytes
    other_public_key = RSA.import_key(other_public_key_pem)
    print("Other participant's public key received.")
    return other_public_key

# Encryption and Decryption Functions

These functions utilize `PyCryptodome`'s `PKCS1_OAEP` for secure asymmetric encryption and decryption using the exchanged public and private keys.

* **`encrypt_message(message, public_key)`:** Encrypts a message using the *recipient's* public key.
* **`decrypt_message(encrypted_message, private_key)`:** Decrypts an encrypted message using *your* private key.

In [4]:
def encrypt_message(message, public_key):
    """
    Encrypts a message using the recipient's public key.
    Message must be bytes. Encrypted output is Base64 encoded for safe network transmission.
    """
    cipher_rsa = PKCS1_OAEP.new(public_key)
    # RSA encryption has size limits. For longer messages, hybrid encryption (AES + RSA) is used.
    # For this simple demo, we'll keep messages short or handle chunking if necessary.
    
    # Encode message to bytes if it's a string
    if isinstance(message, str):
        message = message.encode('utf-8')

    try:
        encrypted_message = cipher_rsa.encrypt(message)
        return base64.b64encode(encrypted_message) # Base64 encode for network transmission
    except ValueError as e:
        print(f"[-] Encryption error: {e}. Message might be too long for direct RSA encryption.")
        print("For real-world apps, use hybrid encryption (e.g., AES for data, RSA for AES key).")
        return None


def decrypt_message(encrypted_message_b64, private_key):
    """
    Decrypts a Base64 encoded message using the recipient's private key.
    """
    cipher_rsa = PKCS1_OAEP.new(private_key)
    try:
        encrypted_message = base64.b64decode(encrypted_message_b64) # Base64 decode
        decrypted_message = cipher_rsa.decrypt(encrypted_message)
        return decrypted_message.decode('utf-8') # Decode bytes back to string
    except ValueError as e:
        print(f"[-] Decryption error: {e}. Check if the correct private key is used or message is corrupted.")
        return None

# Chat Application Logic (Server and Client)

This project requires two separate Python scripts to run simultaneously: a server and a client.

* **`chat_server.py`:** Listens for incoming connections, performs key exchange, and then facilitates encrypted message sending and receiving.
* **`chat_client.py`:** Connects to the server, performs key exchange, and then facilitates encrypted message sending and receiving.

Each script will use threading to handle receiving messages in the background while the user can type and send messages.

# Conceptual Output / Application Screenshots

The chat application runs in separate terminals, below are the screenshots of both the server and client terminals during an active chat session to demonstrate its functionality.

# Server Terminal Output:

![Chat Server Screenshot](chat_server_screenshot.png)

# Client Terminal Output:

![Chat Client Screenshot](chat_client_screenshot.png)

# Conclusion - Project 6: Secure Chat App

This project successfully developed a basic secure chat application leveraging Python's socket programming for network communication and the `PyCryptodome` library for RSA-based end-to-end encryption. By implementing separate server and client components, and integrating key generation, exchange, and message encryption/decryption, the application effectively demonstrates the fundamental principles of secure communication. This hands-on experience provides valuable insight into the importance of cryptography in safeguarding digital interactions and serves as an educational foundation for understanding more complex secure messaging systems.